In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="cricket_ml"
)


In [2]:
query = """
SELECT
    g.ground_name,
    COUNT(DISTINCT bd.match_id) AS matches_played,
    SUM(bd.runs_total) AS total_runs
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
ORDER BY total_runs DESC
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_5296\3957984024.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,matches_played,total_runs
0,Eden Gardens,7,2481.0
1,Rajiv Gandhi International Stadium,6,2130.0
2,Wankhede Stadium,6,2026.0
3,Narendra Modi Stadium,5,2024.0
4,MA Chidambaram Stadium,6,1899.0
5,Sawai Mansingh Stadium,5,1889.0
6,BRSABV Ekana Cricket Stadium,5,1762.0
7,M Chinnaswamy Stadium,5,1687.0
8,Arun Jaitley Stadium,4,1519.0
9,IS Bindra Stadium,3,942.0


In [3]:
query = """
SELECT
    g.ground_name,
    SUM(CASE WHEN bd.is_six = 1 THEN 1 ELSE 0 END) AS sixes
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
ORDER BY sixes DESC
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_5296\214733606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,sixes
0,Eden Gardens,122.0
1,Wankhede Stadium,106.0
2,Rajiv Gandhi International Stadium,104.0
3,Narendra Modi Stadium,103.0
4,Sawai Mansingh Stadium,92.0
5,M Chinnaswamy Stadium,88.0
6,MA Chidambaram Stadium,75.0
7,BRSABV Ekana Cricket Stadium,70.0
8,Arun Jaitley Stadium,63.0
9,HPCA Stadium,49.0


In [4]:
query = """
SELECT
    g.ground_name,
    COUNT(*) AS balls,
    SUM(CASE WHEN bd.is_four = 1 OR bd.is_six = 1 THEN 1 ELSE 0 END) AS boundaries
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
HAVING boundaries > 0
"""
df = pd.read_sql(query, conn)

df["balls_per_boundary"] = df["balls"] / df["boundaries"]
df = df.sort_values("balls_per_boundary")
df


C:\Users\hp\AppData\Local\Temp\ipykernel_5296\727653267.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,balls,boundaries,balls_per_boundary
7,Rajiv Gandhi International Stadium,1309,305.0,4.291803
12,ACA-VDCA Cricket Stadium,463,105.0,4.409524
3,Eden Gardens,1590,347.0,4.582133
5,Sawai Mansingh Stadium,1180,257.0,4.591440
9,HPCA Stadium,569,122.0,4.663934
2,M Chinnaswamy Stadium,1126,236.0,4.771186
4,Arun Jaitley Stadium,1000,209.0,4.784689
11,Narendra Modi Stadium,1271,263.0,4.832700
1,Wankhede Stadium,1400,271.0,5.166052
10,BRSABV Ekana Cricket Stadium,1211,231.0,5.242424


In [5]:
query = """
SELECT
    g.ground_name,
    CASE
        WHEN bd.over_number BETWEEN 0 AND 5 THEN 'Powerplay'
        WHEN bd.over_number BETWEEN 6 AND 14 THEN 'Middle'
        ELSE 'Death'
    END AS phase,
    SUM(bd.runs_total) AS runs
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name, phase
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_5296\4089761523.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,phase,runs
0,MA Chidambaram Stadium,Powerplay,585.0
1,MA Chidambaram Stadium,Middle,819.0
2,MA Chidambaram Stadium,Death,495.0
3,Wankhede Stadium,Powerplay,645.0
4,Wankhede Stadium,Middle,928.0
5,Wankhede Stadium,Death,453.0
6,M Chinnaswamy Stadium,Powerplay,522.0
7,M Chinnaswamy Stadium,Middle,756.0
8,M Chinnaswamy Stadium,Death,409.0
9,Eden Gardens,Powerplay,778.0
